# Image classification with a convlutional neural network

We are still classifying 
We'll start with a simple model linear classifier. As we saw before, this kind of classifier will do poorly in cases in which the data cannot be separated with hyper-planes through the space defined by the data. Still, it will be a good first stab at this, because some of the patterns of using TF will be easier to explain

In [1]:
import os.path as op
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import tensorflow as tf

In [3]:
def read_and_decode(filename):
    filename_queue = tf.train.string_input_producer([filename],
                                                    num_epochs=None)

    reader = tf.TFRecordReader()
    _, serialized_example = reader.read(filename_queue)
    features = tf.parse_single_example(
        serialized_example,
        features={
                'image/height': tf.FixedLenFeature([], tf.int64),
                'image/width': tf.FixedLenFeature([], tf.int64),
                'image/depth': tf.FixedLenFeature([], tf.int64),
                'label': tf.FixedLenFeature([], tf.int64),
                'image/raw': tf.VarLenFeature(tf.string)})

    # Shape elements must be int32 tensors!
    height = tf.cast(features['image/height'], tf.int32)
    width = tf.cast(features['image/width'], tf.int32)
    depth = tf.cast(features['image/depth'], tf.int32)
    
    # Decode the image from its raw representation:
    image = tf.decode_raw(features['image/raw'].values, tf.uint8)

    # Reshape it back to its original shape:
    im_shape = tf.pack([height, width, depth])
    image = tf.reshape(image, im_shape)
    #tf.random_crop(image, [height, width, depth])
    # Convert from [0, 255] -> [0, 1] floats.
    image = tf.cast(image, tf.float32) * (1. / 255)

    # Convert label from a scalar uint8 tensor to an int32 scalar.
    label = tf.cast(features['label'], tf.int32)
    return image, label

In [4]:
image, label = read_and_decode(op.expanduser(op.join('~', 'data_ucsf', 'cells_train.tfrecords')))
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)
tf.train.start_queue_runners(sess=sess)
im_1, lab_1 = sess.run([image, label])

TypeError: Expected string, got list containing Tensors of type '_Message' instead.

In [ ]:
image, label = read_and_decode(op.expanduser(op.join('~', 'data_ucsf', "cells_train.tfrecords")))
images_batch, labels_batch = tf.train.shuffle_batch(
    [image, label], batch_size=40,
    capacity=400,
    shapes=(im_1.shape, lab_1.shape),
    min_after_dequeue=200)

In [ ]:
def weight_variable(name, shape):
    """ 
    Initialize weights with the Xavier initialization
    """
    return tf.get_variable(name, shape=shape, initializer=tf.contrib.layers.xavier_initializer())


def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

In [ ]:
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                          strides=[1, 2, 2, 1], padding='SAME')

In [ ]:
W_conv1 = weight_variable("W_conv1", [5, 5, 3, 32])
b_conv1 = bias_variable([32])

First convolutional layer:

In [ ]:
h_conv1 = tf.nn.relu(conv2d(images_batch, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

Second convolutional layer:

In [ ]:
W_conv2 = weight_variable("W_conv2", [5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

Fully connected layer:

In [ ]:
W_fc1 = weight_variable("W_fc1", [64 * 64 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 64 * 64 * 64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

Dropout:

In [ ]:
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

Readout:

In [ ]:
W_fc2 = weight_variable("W_fc2", [1024, 3])
b_fc2 = bias_variable([3])
y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

In [ ]:
loss = tf.nn.sparse_softmax_cross_entropy_with_logits(y_conv, labels_batch)
loss_mean = tf.reduce_mean(loss)
train_op = tf.train.AdamOptimizer(1e-5).minimize(loss_mean)

In [ ]:
y_pred = tf.cast(tf.argmax(y_conv, 1), tf.int32)

In [ ]:
correct_prediction = tf.equal(y_pred, labels_batch)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [ ]:
# These variables are used for evaluation (helping to decide when to stop training):
image_eval, label_eval = read_and_decode(op.expanduser(op.join('~', 'data_ucsf', "cells_eval.tfrecords")))

# We use a different batch of 40 every time: 
images_eval_batch, labels_eval_batch = tf.train.batch(
            [image_eval, label_eval], batch_size=40,
            shapes=(im_1.shape, lab_1.shape))

# Reproducing the entire network on eval data:
h_conv1_eval = tf.nn.relu(conv2d(images_eval_batch, W_conv1) + b_conv1)
h_pool1_eval = max_pool_2x2(h_conv1_eval)

h_conv2_eval = tf.nn.relu(conv2d(h_pool1_eval, W_conv2) + b_conv2)
h_pool2_eval = max_pool_2x2(h_conv2_eval)

h_pool2_flat_eval = tf.reshape(h_pool2_eval, [-1, 64 * 64 * 64])
h_fc1_eval = tf.nn.relu(tf.matmul(h_pool2_flat_eval, W_fc1) + b_fc1)

y_pred_eval = tf.matmul(h_fc1_eval, W_fc2) + b_fc2

correct_prediction_eval = tf.reduce_mean(
    tf.cast(
        tf.equal(
            tf.cast(
                tf.argmax(y_pred_eval, 1), tf.int32), labels_eval_batch), 
            tf.float32))

In [ ]:
# These will be used for a final test:
image_test, label_test = read_and_decode(op.expanduser(op.join('~', 'data_ucsf', "cells_test.tfrecords")))

# Use the whole thing  
images_test_batch, labels_test_batch = tf.train.batch(
            [image_test, label_test], batch_size=40,
            shapes=(im_1.shape, lab_1.shape))

# Reproducing the entire network on eval data:
h_conv1_test = tf.nn.relu(conv2d(images_test_batch, W_conv1) + b_conv1)
h_pool1_test = max_pool_2x2(h_conv1_test)

h_conv2_test = tf.nn.relu(conv2d(h_pool1_test, W_conv2) + b_conv2)
h_pool2_test = max_pool_2x2(h_conv2_test)

h_pool2_flat_test = tf.reshape(h_pool2_test, [-1, 64 * 64 * 64])
h_fc1_test = tf.nn.relu(tf.matmul(h_pool2_flat_test, W_fc1) + b_fc1)

y_pred_test = tf.matmul(h_fc1_test, W_fc2) + b_fc2

correct_prediction_test = tf.reduce_mean(
    tf.cast(
        tf.equal(
            tf.cast(
                tf.argmax(y_pred_test, 1), tf.int32), labels_test_batch), 
            tf.float32))

In [ ]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)
tf.train.start_queue_runners(sess=sess)

In [ ]:
n_iterations = 0
mean_losses = []
mean_evals = []
max_iterations = 5000

while True:    
    # This is where learning actually takes place!
    _, loss_mean_val = sess.run([train_op, loss_mean], feed_dict={keep_prob: 0.5})
    # Store the loss so we can look at it:
    mean_losses.append(loss_mean_val)
    # Every 10 learning iterations, we consider whether to stop:
    if np.mod(n_iterations, 10) == 0:
        mean_evals.append(sess.run(correct_prediction_eval))
        print("At step %s, mean evaluated accuracy is: %2.2f"%(n_iterations, mean_evals[-1]))
        # But we really only start thinking about stopping 
        # after 2000 iterations:
        if n_iterations > 2000:
            # Here's how we decide whether to keep going, 
            # based on the held-out data:            
            # If you are still improving, relative to recent past keep training:
            if mean_evals[-1] < (np.mean(mean_evals[-10:-1])):
                break

    # If we're still around iterate:
    n_iterations = n_iterations + 1  

    # If you kept going for very long, break anyway:
    if n_iterations > max_iterations:
        break

In [ ]:
fig, ax = plt.subplots(1)
ax.plot(mean_losses)
ax2 = plt.twinx()
ax2.plot(np.arange(0, n_iterations+1, 10), mean_evals, 'ro')

In [ ]:
p = sess.run(correct_prediction_test)

In [ ]:
print(p)